In [56]:
# Install Libraries
# !pip install textblob
# !pip install tweepy

In [57]:
# Import Libraries
from textblob import TextBlob
import tweepy
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [58]:
# Authentication
apiKey = ""
apiSecret = ""
accessToken = "" 
accessTokenSecret = ""
auth = tweepy.OAuthHandler(apiKey, apiSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [59]:
# search tweets
keyword = input("Please enter keyword to search: ")
numOfTweet = int(input ("Please enter the number of tweets to analyze: "))

tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(numOfTweet)
tweet_list = []
for tweet in tweets:
    tweet_list.append(tweet.text)

Please enter keyword to search: red sox
Please enter the number of tweets to analyze: 1500


In [60]:
tweet_list = pd.DataFrame(tweet_list)
print("total number: ",len(tweet_list))

total number:  1500


In [61]:
# delete duplicates
tweet_list.drop_duplicates(inplace = True)
print("total number: ",len(tweet_list))

total number:  944


In [62]:
#Extracting text values
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
tw_list

,0,text
0,"RT @BostonStrongBR: ‼️🔥Hey, Boston Lovers 🔥‼️\...","RT @BostonStrongBR: ‼️🔥Hey, Boston Lovers 🔥‼️\..."
1,@ichirourobles_0 @CantH0LDMe I’m a Red Sox fan...,@ichirourobles_0 @CantH0LDMe I’m a Red Sox fan...
2,RT @MLBONFOX: The Yankees &amp; Red Sox are cu...,RT @MLBONFOX: The Yankees &amp; Red Sox are cu...
3,Added the Yankees / Red Sox Over 9.5 just beca...,Added the Yankees / Red Sox Over 9.5 just beca...
5,RT @krysscarr: you might wanna turn your sound...,RT @krysscarr: you might wanna turn your sound...
...,...,...
1493,RT @andyrivera_23: BUMASS RED SOX GET THE FUCK...,RT @andyrivera_23: BUMASS RED SOX GET THE FUCK...
1494,My preferred end of year outcome is Yankees lo...,My preferred end of year outcome is Yankees lo...
1495,"Unconfirmed but from IG, looks like some Red S...","Unconfirmed but from IG, looks like some Red S..."
1496,Red Sox by a million today https://t.co/CnC58g...,Red Sox by a million today https://t.co/CnC58g...


In [63]:
#Cleaning Text (RT, Punctuation etc)

#Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]

#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head(10)

,0,text
0,"RT @BostonStrongBR: ‼️🔥Hey, Boston Lovers 🔥‼️\...",hey boston lovers te chega a que t ...
1,@ichirourobles_0 @CantH0LDMe I’m a Red Sox fan...,0 i m a red sox fan so i hope they start e...
2,RT @MLBONFOX: The Yankees &amp; Red Sox are cu...,the yankees amp red sox are currently tied ...
3,Added the Yankees / Red Sox Over 9.5 just beca...,added the yankees red sox over 9 5 just beca...
5,RT @krysscarr: you might wanna turn your sound...,you might wanna turn your sound on for this v...
6,@KyJoeCox Like I'm blinded by 2013 Lester.\n\n...,like i m blinded by 2013 lester i fell in ...
7,Patriots lose and the Red Sox try not to get s...,patriots lose and the red sox try not to get s...
8,@MLB @BobbyDalbec @RedSox Tonight’s game IS th...,tonight s game is the season for the red...
9,@MLB @BobbyDalbec @RedSox I won't be able to w...,i won t be able to watch snb but i am de...
10,MLB LOCK OF THE DAY🔒 \nRED SOX vs YANKEES ⚾️\n...,mlb lock of the day red sox vs yankees do...


In [64]:
#Calculating Negative, Positive, Neutral and Compound values

tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tw_list.loc[index, 'sentiment'] = "positive"
    else:
        tw_list.loc[index, 'sentiment'] = "neutral"
    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp

tw_list.head(10)

,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,"RT @BostonStrongBR: ‼️🔥Hey, Boston Lovers 🔥‼️\...",hey boston lovers te chega a que t ...,0.433333,0.733333,positive,0.085,0.656,0.259,0.6705
1,@ichirourobles_0 @CantH0LDMe I’m a Red Sox fan...,0 i m a red sox fan so i hope they start e...,0.100000,0.150000,positive,0.000,0.666,0.334,0.6150
2,RT @MLBONFOX: The Yankees &amp; Red Sox are cu...,the yankees amp red sox are currently tied ...,0.033333,0.266667,neutral,0.000,1.000,0.000,0.0000
3,Added the Yankees / Red Sox Over 9.5 just beca...,added the yankees red sox over 9 5 just beca...,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
5,RT @krysscarr: you might wanna turn your sound...,you might wanna turn your sound on for this v...,0.300000,0.466667,positive,0.098,0.789,0.113,0.1027
6,@KyJoeCox Like I'm blinded by 2013 Lester.\n\n...,like i m blinded by 2013 lester i fell in ...,0.250000,0.300000,positive,0.000,0.649,0.351,0.8481
7,Patriots lose and the Red Sox try not to get s...,patriots lose and the red sox try not to get s...,0.000000,0.000000,negative,0.197,0.803,0.000,-0.4019
8,@MLB @BobbyDalbec @RedSox Tonight’s game IS th...,tonight s game is the season for the red...,0.033333,0.383333,positive,0.000,0.693,0.307,0.7964
9,@MLB @BobbyDalbec @RedSox I won't be able to w...,i won t be able to watch snb but i am de...,0.433333,0.341667,positive,0.124,0.476,0.399,0.7351
10,MLB LOCK OF THE DAY🔒 \nRED SOX vs YANKEES ⚾️\n...,mlb lock of the day red sox vs yankees do...,-0.200000,0.200000,negative,0.086,0.914,0.000,-0.1531


In [65]:
#Creating new data frames for all sentiments (positive, negative and neutral)

tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

In [66]:
#Function for count_values_in single columns

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [67]:
#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

,Total,Percentage
positive,421,44.60
neutral,356,37.71
negative,167,17.69
